In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
from wordcloud import STOPWORDS
import re

In [2]:
#Only do once if needed (error will show in removing of stopwords when needed)
#nltk.download('stopwords')

In [3]:
# Loading the data 
path = 'C:\\Users\\jjehm\\OneDrive\\Dokumente\\Uni\\05_SS 19\\Masterarbeit\\Daten\\Cleaned\\all_gr50.csv'
df = pd.read_csv(path, dtype={"kindle":object})
df.drop(df.columns[0],axis=1, inplace=True)
figpath = 'C://Users//jjehm//OneDrive//Dokumente//Uni//05_SS 19//Masterarbeit//01_Arbeit//LaTex//Figures//Pandas Figures//'
df['kindle'] = df['kindle'].astype(str)
df = df[df['kindle'].isin(["paperwhite 7", "8", "oasis 9","paperwhite 10"])]# 
df.reset_index(inplace=True, drop=True)

In [5]:
%run func_bi_dict_analysis.ipynb

The function bi_dict_analysis is now available.
 Pass in a data frame and the name of the dicionary (as string with .xlsx) as arguments
The dataframe 'df' should be limited to the rows you want to analyze
Review text and title will be added in the reviews column
words will be: lowered, stop words removed, punctuation and whitespace removed
Returns a tuple with the data frame with the added dictionary columns first and column names second


### Binary dictionary analysis

In [6]:
(da, column_names) = bi_dict_analysis(df,"Hardware_features_dict.xlsx")

Process started:
Status: 5.0%
Status: 5.0%
Status: 10.0%
Status: 10.0%
Status: 20.0%
Status: 40.0%
Status: 80.0%
Analysis successfull


In [7]:
rdf= da.copy()
grdf= rdf.groupby(["kindle"]).sum()
count = rdf.groupby(["kindle"]).count()
count.drop(count.columns[1:],axis=1,inplace=True)
grdf.drop(grdf.columns[0:3],axis=1,inplace=True)
results = grdf
#grdf


In [14]:
latex = grdf.T

#moving the paperwhite 7 column to the front for proper order
latex = latex[ ['paperwhite 7'] + [ col for col in latex.columns if col != 'paperwhite 7' ] ]
latex = latex.astype(int)

print (latex.to_latex())

\begin{tabular}{lrrrr}
\toprule
kindle &  paperwhite 7 &     8 &  oasis 9 &  paperwhite 10 \\
\midrule
Price                &          1085 &   552 &      363 &            244 \\
Waterproof           &            57 &    15 &      198 &            388 \\
Capacity             &            81 &    26 &       54 &            135 \\
Display              &          3015 &  1172 &      683 &            674 \\
Resolution           &           331 &    44 &       56 &             39 \\
Light                &          4043 &  1318 &      516 &            832 \\
Warm light           &             2 &     1 &        1 &              1 \\
Automatic brightness &           453 &   120 &       71 &             83 \\
Page turn buttons    &           714 &   298 &      403 &            231 \\
Material             &           111 &    55 &       96 &             32 \\
Connectivity         &           407 &   121 &       80 &            122 \\
\bottomrule
\end{tabular}



In [9]:
#count

In [10]:
#calculates the mean rating whether the topic was discussed about or not
results = pd.DataFrame()
for column in column_names:
    zdf = rdf.groupby([column]).mean()
    zdf.drop(zdf.columns[1:], axis = 1, inplace = True)
    results[column] = zdf.stars

#results  

In [11]:
#calculates how many times the topics were mentioned across the kindles
df = rdf
grdf= rdf.groupby(["kindle"]).sum()
count = rdf.groupby(["kindle"]).count()
count.drop(count.columns[1:],axis=1,inplace=True)
grdf.drop(grdf.columns[0:3],axis=1,inplace=True)

#grdf.sum()

### Ratings dependend on discussed topic

In [15]:
#generates the table for the latex document
table = results.T
table["Num. of reviews"] = list(grdf.sum())
table["Num. of reviews"] = table["Num. of reviews"].astype(int)
table["change"] = round(table[1.0]-table[0.0],2)
table[1.0] = round(table[1.0],2)
table[0.0] = round(table[0.0],2)
table = table[["Num. of reviews", 1.0, 0.0, "change"]]

def redgreen(num):
    if num<0:
        return "\textcolor{DarkRed}{"+str(num)+"}"
    if num>0:
        return "\textcolor{Green}{"+str(num)+"}"
    else:
        return str(num)

table.change = table.change.apply(lambda x: redgreen(x))
print(table.to_latex(escape=False))

\begin{tabular}{lrrrl}
\toprule
Price &  Num. of reviews &   1.0 &   0.0 &                      change \\
\midrule
Price                &             2244 &  3.65 &  3.84 &  \textcolor{DarkRed}{-0.19} \\
Waterproof           &              658 &  4.15 &  3.80 &     \textcolor{Green}{0.35} \\
Capacity             &              296 &  4.11 &  3.81 &      \textcolor{Green}{0.3} \\
Display              &             5544 &  3.84 &  3.80 &     \textcolor{Green}{0.04} \\
Resolution           &              470 &  4.05 &  3.81 &     \textcolor{Green}{0.24} \\
Light                &             6709 &  4.25 &  3.46 &     \textcolor{Green}{0.79} \\
Warm light           &                5 &  4.20 &  3.81 &     \textcolor{Green}{0.39} \\
Automatic brightness &              727 &  3.98 &  3.80 &     \textcolor{Green}{0.18} \\
Page turn buttons    &             1646 &  3.74 &  3.82 &  \textcolor{DarkRed}{-0.08} \\
Material             &              294 &  3.87 &  3.81 &     \textcolor{Green}{0.06

In [13]:
#rdf.stars.mean()